In [1]:
% matplotlib inline
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
db = MySQLdb.connect(host="crunchbase.ckfkhzm7cot3.us-west-2.rds.amazonaws.com",
                     port=3306,
                     user="root",         # your username
                     passwd="root1234",  # your password
                     db="analytics_2")        # name of the data base


In [3]:
cur = db.cursor()

In [4]:
count = cur.execute("SELECT count(*) FROM cb_objects")

#### Load into pandas dataframe from database

In [5]:
df_mysql = pd.read_sql('select * from cb_objects LIMIT 100;', con=db)

In [19]:
persons = pd.read_sql('Select * from cb_objects where entity_type = \'Person\'', con=db)

In [20]:
relationships = pd.read_sql('Select * from cb_relationships', con=db)

In [21]:
degrees = pd.read_sql('Select * from cb_degrees', con=db)

In [22]:
funding_rounds = pd.read_sql('Select * from cb_funding_rounds', con=db)

In [23]:
baseline_data = pd.read_sql("Select * from cb_master_data_2", con=db)

In [24]:
companies = pd.read_sql('Select * from cb_objects where entity_type = \'Company\'', con=db)

In [234]:
funding_round_order = pd.read_sql('select * from cb_funding_round_order', con=db)

In [ ]:
degrees = baseline_data

#### Fill nan for empty and None values

In [ ]:
degrees.fillna(value=np.nan, inplace=True)

In [25]:
persons.fillna(value=np.nan, inplace=True)
relationships.fillna(value=np.nan, inplace=True)
degrees.fillna(value=np.nan, inplace=True)

#### People

For the baseline model we need to consider only people who are Founders, CEO, CIO etc

#### Work Experience

In [ ]:
len(persons)

In [26]:
work_ex = pd.merge(persons, relationships, how='inner', right_on='person_object_id', left_on='id')

In [27]:
print 'Total number of work ex rows', len(work_ex)

Total number of work ex rows 401957


In [28]:
workex_with_no_persons = work_ex[work_ex['id_x'].isnull()]
person_with_no_workex = work_ex[work_ex['id_y'].isnull()]
print "workex rows with no corresponding person",len(workex_with_no_persons)
print "Persons with no details about their workex",len(person_with_no_workex)

workex rows with no corresponding person 0
Persons with no details about their workex 0


In [29]:
print "Number of rows with no start_at for workex is: ",len(work_ex[work_ex['start_at'].isnull()]), "which is roughly half the number of rows"

Number of rows with no start_at for workex is:  195122 which is roughly half the number of rows


Idea : We can fill the start_at date with one of the following:
1. founding_date of the company
2. created_at_y which is when the record was created in the database

To analyse we can first build a plot of the different between the 2 and see the relation between the title, created at and founding date. It can mean that most of the founders have their profile created on crunchbase much later than the founding date, in which case we can safely use the founding date. However, for employess we an use the created_at.

In [30]:
work_ex['title'].unique

<bound method Series.unique of 0                         Co-Founder/CEO/Board of Directors
1                                           Co-founder & VP
2                                      Business Development
3                                           Limited Partner
4                                                Consultant
5                                                Consultant
6                                                Consultant
7                                                Consultant
8                                              VP Marketing
9                                           Limited Partner
10                                                         
11                                                         
12                                                         
13                                                         
14                                                         
15                                                         
16       

This below line of code filters all the work ex where the person is one of the founding members or the board of directors

In [31]:
len(work_ex)

401957

In [32]:
founders = work_ex.loc[work_ex['id_x'].isin(work_ex.loc[work_ex['title'].str.contains('Cofounder', regex=True),'id_x'])]

In [17]:
founders['title']

1625                                          Board
1626                                      Cofounder
3454      CoFounder & Vice President of Engineering
3455                                Cofounder & CEO
3456                                Product Manager
3457                                            CEO
3458                                Cofounder & CEO
3459                                        Advisor
3460                                        Advisor
3461                                        Advisor
4270                                     NY Partner
4271                                   Board Member
4272                                      Cofounder
4273                               Business Advisor
4274                               Managing Partner
4275                                            EVP
4276                        Regional Advisory Board
4277       Co-Chairman & Co-Chief Executive Officer
4278                                 Advisory Board
4279        

In [33]:
founders = work_ex.loc[work_ex['id_x'].isin(work_ex.loc[work_ex['title'].str.contains('Chief|C[A-Z]O|Co\-Founder|Co\-founder|Board', regex=True),'id_x'])]

In [34]:
founders['start_at'] = pd.to_datetime(founders['start_at'])
# import dateutil
# founderspd.to_datetime(founders['start_at'])
# founders['start_at']
# len(founders[founders['start_at'].isnull()])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
founders.groupby('id_x').min()#agg({'start_at':min})

In [ ]:
founder_groups = founders.groupby('id_x')#.agg({'start_at':pd.datetime})
founder_groups['start_at'].min()
founders[founders['id_x']=='p:99969']

In [ ]:
import datetime
# pd.to_datetime(founders['start_at'])[5] > pd.to_datetime('2012-01-01')
np.min(pd.to_datetime(founders['start_at']))

TODO: Aggregate as the min of teh start_at, this will give the first date when work ex added

In [ ]:
len(founders['relationship_object_id'].unique())

In [ ]:
funding_rounds

In [35]:
tabl = pd.merge(founders, funding_rounds,how='inner', left_on='relationship_object_id', right_on='object_id')

In [36]:
tabl[['funding_round_id','name','title']]

,funding_round_id,name,title
0,888,Ben Elowitz,Co-Founder/CEO/Board of Directors
1,889,Ben Elowitz,Co-Founder/CEO/Board of Directors
2,2312,Ben Elowitz,Co-Founder/CEO/Board of Directors
3,888,Gus Tai,Board Member
4,889,Gus Tai,Board Member
5,2312,Gus Tai,Board Member
6,888,Theresia Ranzetta,Board
7,889,Theresia Ranzetta,Board
8,2312,Theresia Ranzetta,Board
9,888,Alex Berg,VP User Experience


#### Education

In [37]:
len(degrees)

109610

In [38]:
degrees['degree_type'].dtype

dtype('O')

In [39]:
len(degrees['object_id'].unique())

68451

We have degree details for the above number of people

We need to give a rating to the degrees. All Bachelors will have rating as 1. Masters will have rating 2 and PHD/MBA will have rating 3.As there are around 7000 differnet degrees, for other types of degrees we can use the median of the rest of the table. 

In [40]:
degrees.loc[degrees['degree_type'].str.contains('B\..*',na=False),'degree_score'] = 1

In [41]:
degrees.loc[degrees['degree_type'].str.contains('Bachelor',na=False),'degree_score'] = 1

In [42]:
degrees.loc[degrees['degree_type'].str.contains('^B.*',na=False),'degree_score'] = 1

In [43]:
degrees.loc[degrees['degree_type'].str.contains('JD',na=False),'degree_score'] = 1

In [44]:
degrees.loc[degrees['degree_type'].str.contains('Master',na=False),'degree_score'] = 2

In [45]:
degrees.loc[degrees['degree_type'].str.contains('^M.*',na=False),'degree_score'] = 2

In [46]:
degrees.loc[degrees['degree_type'].str.contains('M.$',na=False),'degree_score'] = 2

In [47]:
degrees.loc[degrees['degree_type'].str.contains('M\..*',na=False),'degree_score'] = 2

In [48]:
degrees.loc[degrees['degree_type'].str.contains('Ph\.*D',na=False),'degree_score'] = 3

In [49]:
degrees.loc[degrees['degree_type'].str.contains('MBA',na=False),'degree_score'] = 2

In [50]:
degrees.loc[degrees['degree_type'].str.contains('No degree',na=False),'degree_score'] = 0

In [51]:
degrees.loc[degrees['degree_type'].str.contains('LLB',na=False),'degree_score'] = 1

In [52]:
degrees.loc[degrees['degree_type'].str.contains('Foundation',na=False),'degree_score'] = 0.5

In [53]:
degrees.loc[degrees['degree_type'] == '','degree_score'] = 1

In [54]:
degrees['degree_score'].fillna(np.nanmedian(degrees['degree_score']), inplace=True)

In [55]:
degrees[['degree_type','degree_score']]

,degree_type,degree_score
0,MBA,2.0
1,BA,1.0
2,MS,2.0
3,MS,2.0
4,BCS,1.0
5,BS,1.0
6,BS,1.0
7,MS,2.0
8,BA,1.0
9,BS,1.0


In [56]:
degrees.loc[degrees['degree_score'].isnull()]['degree_type'].describe()

count     0
unique    0
Name: degree_type, dtype: int64

In [57]:
len(degrees)

109610

In [58]:
degrees.drop_duplicates(subset=['object_id','degree_type'],inplace=True)

In [59]:
baseline_data = pd.merge(baseline_data, degrees, how='left', left_on=['object_id','degree_type'], right_on=['object_id','degree_type'])

In [60]:
baseline_data['degree_score'].fillna(np.nanmedian(baseline_data['degree_score']), inplace=True)

In [61]:
len(baseline_data)

99327

In [62]:
len(baseline_data)

99327

In [63]:
# education[['object_id','degree_score','degree_type']]
baseline_data[['degree_score','degree_type']]

,degree_score,degree_type
0,1.0,
1,1.0,
2,1.0,
3,1.0,None
4,1.0,None
5,1.0,None
6,1.0,None
7,1.0,
8,0.0,No degree
9,1.0,BS


In [64]:
education_with_no_persons = baseline_data[baseline_data['id_x'].isnull()]
person_with_no_education = baseline_data[baseline_data['id_y'].isnull()]
print "Education rows with no corresponding person",len(education_with_no_persons)
print "Persons with no details about their education",len(person_with_no_education)

Education rows with no corresponding person 0
Persons with no details about their education 54265


In [65]:
education_with_no_persons = education[education['id_x'].isnull()]
person_with_no_education = education[education['id_y'].isnull()]
print "Education rows with no corresponding person",len(education_with_no_persons)
print "Persons with no details about their education",len(person_with_no_education)

NameError: name 'education' is not defined

Next Steps: 
1. We have sparse data for education. What to do with it?
2. How to clean workex data?

#### Funding Rounds

In [66]:
len(funding_rounds)

52928

In [67]:
work_ex_funding = pd.merge(work_ex, funding_rounds,how='inner', left_on='relationship_object_id', right_on='object_id')

In [68]:
len(work_ex_funding)

312447

In [69]:
work_ex_funding[:5]
funding_rounds['funded_at_datetime']= pd.to_datetime(funding_rounds['funded_at'])
funding_rounds[['funded_at_datetime','funded_at']]

,funded_at_datetime,funded_at
0,2006-12-01,2006-12-01
1,2004-09-01,2004-09-01
2,2005-05-01,2005-05-01
3,2006-04-01,2006-04-01
4,2006-05-01,2006-05-01
5,2007-01-01,2007-01-01
6,2007-03-01,2007-03-01
7,2005-12-01,2005-12-01
8,2007-05-01,2007-05-01
9,2006-06-01,2006-06-01


In [70]:
kevin = work_ex_funding[work_ex_funding['id_x']=='p:3']
kevin[['title','funded_at']]
kevin

,id_x,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round,source_url,source_description,created_by_y,created_at,updated_at
3,p:3,Person,3,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,NaN,None,2,0,1,http://seattlepi.nwsource.com/business/246734_...,None,initial-importer,2007-11-07 01:01:43,2008-05-19 03:39:17
4,p:3,Person,3,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,NaN,None,3,0,0,http://pulse2.com/2007/01/09/wiki-builder-webs...,None,initial-importer,2007-11-07 01:05:01,2008-05-19 03:39:17
5,p:3,Person,3,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,NaN,None,4,1,0,http://www.accel.com/news/news_one_up.php?news...,Accel,mike,2008-05-19 03:39:17,2008-11-04 04:24:30
213,p:3,Person,3,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,NaN,None,0,0,1,None,None,None,2012-01-07 18:44:26,2012-01-15 01:55:50
214,p:3,Person,3,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,NaN,None,4,1,0,http://www.salesportal.com/public-site/pdf/pre...,Sales Portal Secures Equity Investment,None,2012-01-07 19:11:39,2012-01-09 23:31:53
237,p:3,Person,3,NaN,Kevin Flaherty,kevin flaherty,/person/kevin-flaherty,NaN,operating,NaN,...,NaN,None,1,1,1,http://www.finsmes.com/2012/10/ritani-raises-1...,Ritani Raises $15M in Funding,timircha,2012-10-12 03:53:12,2012-10-12 03:53:12


In [ ]:
kevin1 = work_ex[work_ex['id_x']=='p:3']
# kevin1[['title','funded_at']]
# kevin1['start_at_datetime'] = pd.to_datetime(kevin['start_at'].dropna(),yearfirst = True)
# pd.to_datetime(kevin['end_at'])
# pd.to_datetime(kevin['end_at'],infer_datetime_format=True)
# kevin1[['start_at', 'end_at','start_at_datetime', 'end_at_datetime']]
# kevin1[['start_at', 'end_at']]
# pd.to_datetime(kevin1['start_at'].dropna())

In [ ]:
a = '2001-02-01'
pd.to_datetime(a)

In [ ]:
a = work_ex[:5].groupby('id_x')
work_ex[:10]

### Train Test and Evaluation Data

We need to divide the data into train test and evaluation data. We will divide it in the following ratio:

1.Train - 60%

2.Test - 20%

3.Evaluation - 20%


### Baseline Model

Our baseline model is dependent on Education and work experience of the founder. We will train our linear regression model on the score of the founder based on the above 2 features and run it on the test data. 

We will then find the error rate of our baseline model on the test data.

In [71]:
baseline_data['work_ex_score'] = baseline_data['funded_at'] - baseline_data['start_at']

In [72]:
baseline_data = pd.merge(companies, baseline_data, how='inner', left_on ='id', right_on='id_x')

In [73]:
negative_dates = baseline_data.loc[baseline_data['work_ex_score']<pd.Timedelta('0 days')]

In [74]:
len(negative_dates)

11284

In [192]:
baseline_data[['funded_at','start_at','work_ex_score']]

,funded_at,start_at,work_ex_score
0,2005-10-01,1973-01-01,11961 days
1,2005-10-01,2005-10-17,0 days
2,2005-10-01,1901-01-01,38259 days
3,2005-10-01,2007-01-01,0 days
4,2007-01-01,1973-01-01,12418 days
5,2007-01-01,2005-10-17,441 days
6,2007-01-01,1901-01-01,38716 days
7,2007-01-01,2007-01-01,0 days
8,2008-05-19,1973-01-01,12922 days
9,2008-05-19,2005-10-17,945 days


In [76]:
for index,row in negative_dates.iterrows():
#     print row['funded_at'], companies.loc[companies['id']== row['id_x'],'founded_at']
#     print row['funded_at'] - companies.loc[companies['id']== row['id_x'],'founded_at']
#     print row['work_ex_score']
    if  row['funded_at'] and row['founded_at']:
        baseline_data.loc[index,'work_ex_score'] = row['funded_at'] - row['founded_at']
        row['work_ex_score']
#     baseline_data.loc[baseline_data['object_id']==row['object_id'],'work_ex_score'] = row['work_ex_score']
#     print baseline_data.loc[baseline_data['object_id']==row['object_id'],'work_ex_score']

In [77]:
len(baseline_data)

99327

In [78]:
 baseline_data.loc[baseline_data['work_ex_score']<pd.Timedelta('0 days'),'work_ex_score'] = 0

In [79]:
baseline_data.columns

Index([u'id', u'entity_type', u'entity_id', u'parent_id', u'name_x',
       u'normalized_name', u'permalink', u'category_code', u'status',
       u'founded_at', u'closed_at', u'domain', u'homepage_url',
       u'twitter_username', u'logo_url', u'logo_width', u'logo_height',
       u'short_description', u'description', u'overview', u'tag_list',
       u'country_code', u'state_code', u'city', u'region',
       u'first_investment_at', u'last_investment_at', u'investment_rounds',
       u'invested_companies', u'first_funding_at', u'last_funding_at',
       u'funding_rounds', u'funding_total_usd', u'first_milestone_at',
       u'last_milestone_at', u'milestones', u'relationships', u'created_by',
       u'created_at_x', u'updated_at_x', u'degree_type', u'object_id',
       u'start_at', u'title', u'first_name', u'last_name', u'name_y', u'id_x',
       u'funding_round_id', u'funded_at', u'funding_round_code',
       u'funding_round_type', u'data_class', u'id_y', u'subject',
       u'institutio

In [80]:
baseline_data.columns#.drop_duplicates(inplace=True)

Index([u'id', u'entity_type', u'entity_id', u'parent_id', u'name_x',
       u'normalized_name', u'permalink', u'category_code', u'status',
       u'founded_at', u'closed_at', u'domain', u'homepage_url',
       u'twitter_username', u'logo_url', u'logo_width', u'logo_height',
       u'short_description', u'description', u'overview', u'tag_list',
       u'country_code', u'state_code', u'city', u'region',
       u'first_investment_at', u'last_investment_at', u'investment_rounds',
       u'invested_companies', u'first_funding_at', u'last_funding_at',
       u'funding_rounds', u'funding_total_usd', u'first_milestone_at',
       u'last_milestone_at', u'milestones', u'relationships', u'created_by',
       u'created_at_x', u'updated_at_x', u'degree_type', u'object_id',
       u'start_at', u'title', u'first_name', u'last_name', u'name_y', u'id_x',
       u'funding_round_id', u'funded_at', u'funding_round_code',
       u'funding_round_type', u'data_class', u'id_y', u'subject',
       u'institutio

In [130]:
baseline_data.fillna(np.NaN, inplace=True)
# baseline_data[baseline_data['closed_at']==np.NaN]
closed_rows = baseline_data[pd.notnull(baseline_data['closed_at'])]
closed_rows['class'] = 1
current_companies = baseline_data[pd.isnull(baseline_data['closed_at'])]
current_companies['class'] = 0

/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [275]:
negative_samples = baseline_data.loc[pd.notnull(baseline_data['closed_at'])]
baseline_data.loc[pd.isnull(baseline_data['closed_at']),'class'] = 0

In [276]:
negative_samples.loc[:,'class'] = 1
baseline_data.loc[:,'class'] = 0

In [277]:
negative_samples = negative_samples.reset_index()

In [305]:
for i, row in negative_samples.iterrows():
    code = funding_round_order[(funding_round_order['funding_round_code']==row['funding_round_code']).astype(bool) & 
                                 (funding_round_order['funding_round_type']==row['funding_round_type']).astype(bool)]
    negative_samples.set_value(i,'funding_round_code', code.iloc[0,2])

In [308]:
negative_samples[['name_x','funding_round_code']]#['funding_round_code']
# baseline_data.loc[pd.notnull(baseline_data['closed_at'])]['funding_round_code']

,name_x,funding_round_code
0,Omnidrive,series-a
1,Omnidrive,series-a
2,Wis.dm,series-b
3,Wis.dm,series-b
4,Babelgum,series-b
5,Babelgum,series-b
6,Video Passports,series-a
7,Thoof,series-a
8,Wesabe,series-a
9,Wesabe,series-a


In [310]:
baseline_data = pd.concat([baseline_data, negative_samples])

In [311]:
len(baseline_data)

103183

In [146]:
a = companies[pd.notnull(companies['closed_at'])]['last_funding_at']

In [155]:
len(baseline_data[baseline_data['class']==1]['name_x'].unique())

1276

In [ ]:
pd.concat()

In [148]:
companies[pd.notnull(a)]

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


IndexingError: Unalignable boolean Series key provided

In [316]:
final_data  = baseline_data[['name_x','work_ex_score','degree_score',u'funding_round_code',
               'first_name','last_name','funded_at','class']]#.groupby('funded_at').sum()

In [317]:
final_data =  final_data.reset_index()
final_data

,index,name_x,work_ex_score,degree_score,funding_round_code,first_name,last_name,funded_at,class
0,0,Wetpaint,11961 days,1.0,a,Ben,Elowitz,2005-10-01,0
1,1,Wetpaint,0 days,1.0,a,Michael,Howell,2005-10-01,0
2,2,Wetpaint,38259 days,1.0,a,Brian,Watkins,2005-10-01,0
3,3,Wetpaint,0 days,1.0,a,Bert,Hogue,2005-10-01,0
4,4,Wetpaint,12418 days,1.0,b,Ben,Elowitz,2007-01-01,0
5,5,Wetpaint,441 days,1.0,b,Michael,Howell,2007-01-01,0
6,6,Wetpaint,38716 days,1.0,b,Brian,Watkins,2007-01-01,0
7,7,Wetpaint,0 days,1.0,b,Bert,Hogue,2007-01-01,0
8,8,Wetpaint,12922 days,1.0,c,Ben,Elowitz,2008-05-19,0
9,9,Wetpaint,945 days,1.0,c,Michael,Howell,2008-05-19,0


In [318]:
final_data.drop_duplicates(inplace=True)

In [319]:
final_data['work_ex_score'].fillna(0,inplace=True)

In [320]:
final_data['work_ex_score_i'] = (final_data['work_ex_score']/ np.timedelta64(1, 'D')).astype(int)

In [321]:
final_data['work_ex_score_i'] = np.ceil(final_data['work_ex_score_i']/365)

In [322]:
final_data

,index,name_x,work_ex_score,degree_score,funding_round_code,first_name,last_name,funded_at,class,work_ex_score_i
0,0,Wetpaint,11961 days,1.0,a,Ben,Elowitz,2005-10-01,0,33.0
1,1,Wetpaint,0 days,1.0,a,Michael,Howell,2005-10-01,0,0.0
2,2,Wetpaint,38259 days,1.0,a,Brian,Watkins,2005-10-01,0,105.0
3,3,Wetpaint,0 days,1.0,a,Bert,Hogue,2005-10-01,0,0.0
4,4,Wetpaint,12418 days,1.0,b,Ben,Elowitz,2007-01-01,0,35.0
5,5,Wetpaint,441 days,1.0,b,Michael,Howell,2007-01-01,0,2.0
6,6,Wetpaint,38716 days,1.0,b,Brian,Watkins,2007-01-01,0,107.0
7,7,Wetpaint,0 days,1.0,b,Bert,Hogue,2007-01-01,0,0.0
8,8,Wetpaint,12922 days,1.0,c,Ben,Elowitz,2008-05-19,0,36.0
9,9,Wetpaint,945 days,1.0,c,Michael,Howell,2008-05-19,0,3.0


In [323]:
data = final_data.groupby(['name_x','funding_round_code'])[['degree_score','work_ex_score_i','class']].sum()

In [324]:
len(data)

34148

In [325]:
data = data.reset_index(1)

In [326]:
len(final_data['name_x'].unique())

20805

In [327]:
len(baseline_data['name_x'].unique())

20805

In [328]:
len(baseline_data)

103183

In [329]:
data

,funding_round_code,degree_score,work_ex_score_i,class
name_x,,,,
#waywire,seed,3.0,30.0,0
(In)Touch Network,seed,1.0,0.0,0
+n (PlusN),seed,2.0,10.0,0
1000 Markets,seed,2.0,2.0,0
1000memories,a,4.0,4.0,0
1000memories,seed,4.0,0.0,0
1001 Menus,a,3.0,9.0,0
100Plus,seed,3.0,8.0,0
100Plus,unattributed,3.0,8.0,0


In [330]:
data['funding_round_category'] = data['funding_round_code'].astype('category').cat.codes
positive = data[data['class']==0]
negative = data[data['class']==1]

In [332]:
def split_data(data):
    msk = np.random.rand(len(data)) < 0.8
    train_data = data[msk]
#     test = data[~msk]
#     msk = np.random.rand(len(test)) < 0.5
#     evaluation_data = test[msk]
    test_data = data[~msk]
    return train_data,test_data

In [333]:
from sklearn import svm

In [334]:

clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

for i in xrange(5):
    train_data, test_data = split_data(positive)
    clf.fit(train_data[['degree_score','work_ex_score_i','funding_round_category']])
#     y_pred_train = clf.predict(train_data[['degree_score','work_ex_score_i','funding_round_category']])
    y_pred_test = clf.predict(test_data[['degree_score','work_ex_score_i','funding_round_category']])
    print "Train data ", len(train_data)
    print "Test data ", len(test_data)
    print "Error ", float(len(y_pred_test[y_pred_test==-1]))/float(len(y_pred_test))

Train data  26048
Test data  6511
Error  0.175088312087
Train data  26078
Test data  6481
Error  0.152908501774
Train data  25936
Test data  6623
Error  0.153253812472
Train data  26042
Test data  6517
Error  0.161730857757
Train data  25997
Test data  6562
Error  0.145839683023


In [335]:
y_pred_test = clf.predict(negative[['degree_score','work_ex_score_i','funding_round_category']])
print "Error ", float(len(y_pred_test[y_pred_test==-1]))/float(len(y_pred_test))

Error  0.854280510018


In [ ]:
len(y_pred_train[y_pred_train==-1])
len(y_pred_train)

In [ ]:
y_pred_test = clf.predict(test_data[['degree_score','work_ex_score_i','funding_round_category']])

In [ ]:
len(y_pred_test[y_pred_test==-1])
len(y_pred_test)

In [ ]:
y_pred_eval = clf.predict(evaluation_data[['degree_score','work_ex_score_i','funding_round_category']])

In [ ]:
len(y_pred_eval[y_pred_eval==-1])
len(y_pred_eval)

In [ ]:
len(data)

In [ ]:
persons.columns

In [ ]:
companies.fillna(np.nan)

In [ ]:
closed_companies = companies[companies['closed_at'].notnull()]

In [ ]:
pd.merge(funding_rounds, closed_companies, left_on='object_id', right_on='id')['object_id'].unique()

#### 2 Class SVM

In [336]:
from sklearn.svm import SVC
clf = SVC()

In [341]:

p = np.random.permutation(len(negative))
new_test_data = pd.concat([positive.iloc[p], negative])


In [348]:
for i in xrange(5):
    train_data, test_data = split_data(new_test_data)
    clf.fit(train_data[['degree_score','work_ex_score_i','funding_round_category']], train_data['class'])
#     y_pred_train = clf.predict(train_data[['degree_score','work_ex_score_i','funding_round_category']])
    y_pred_test = clf.predict(test_data[['degree_score','work_ex_score_i','funding_round_category']])
    print "Train data ", len(train_data)
    print "Test data ", len(test_data)
    error =  sum(abs(y_pred_test - test_data['class']))
    print "Error ", float(error/float(len(y_pred_test)))

Train data  870
Test data  228
Error  0.0394736842105
Train data  886
Test data  212
Error  0.0471698113208
Train data  880
Test data  218
Error  0.0779816513761
Train data  874
Test data  224
Error  0.0446428571429
Train data  887
Test data  211
Error  0.0568720379147


### Findings

In [15]:
companies['category_code'].unique()

array(['web', 'enterprise', 'software', 'news', 'social',
       'network_hosting', 'games_video', 'music', 'mobile', 'search',
       'advertising', 'messaging', 'security', 'photo_video', 'finance',
       'hardware', 'public_relations', 'ecommerce', 'travel', 'other',
       'medical', 'real_estate', 'semiconductor', 'consulting',
       'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech',
       'education', 'transportation', 'manufacturing', 'hospitality',
       'fashion', 'nanotech', 'nonprofit', 'automotive', None,
       'government', 'design', 'local', 'pets'], dtype=object)

In [18]:
len(companies[companies['founded_at'].isnull()])

105326